# Import libraries

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
from scipy import stats

# Prepare datasets

In [3]:
df_x = pd.read_csv('../data/dataset_01/data.csv')
df_y = pd.read_csv('../data/dataset_01/answer.csv')

In [4]:
df_x

In [5]:
df_y

In [6]:
X_train, X_rem, y_train, y_rem = train_test_split(df_x, df_y, train_size=0.8)

X_valid, X_test, y_valid, y_test = train_test_split(X_rem,
                                                    y_rem,
                                                    test_size=0.5)

In [7]:
print(X_train.shape), print(y_train.shape)
print(X_valid.shape), print(y_valid.shape)
print(X_test.shape), print(y_test.shape)

# Some diagrams

In [8]:
# вышло бесполезно пока, но я попробовал
c_mat = df_y.corr()
fir = plt.figure(figsize=(15, 15))

sb.heatmap(c_mat, vmax=.8, square=True)
plt.show()

# First NN (preceptron)

In [78]:
model = keras.Sequential([
    keras.Input(shape=(672, 1)),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(6, activation='linear'),
])

In [70]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.MeanSquaredError(),
              metrics=[keras.metrics.MeanSquaredError()])

In [71]:
checkpoint_name = 'weights/Weights-{epoch:03d}--{val_loss:.5f}.hdf5'
checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_name,
                                             monitor='val_loss',
                                             verbose=1,
                                             save_best_only=True,
                                             mode='auto')
callbacks_list = [checkpoint]

In [20]:
history = model.fit(X_train,
                    y_train,
                    batch_size=256,
                    epochs=50,
                    verbose=0,
                    validation_data=(X_valid, y_valid))  # ,
# callbacks=callbacks_list)

In [21]:
history.history

In [25]:
sb.scatterplot(
    {'loss': history.history['loss'], 'validation loss': history.history['val_loss']})
plt.ylabel("Значение функции потерь")
plt.xlabel("Номер эпохи")

In [23]:
sb.scatterplot({'mean_squared_error': history.history['mean_squared_error'],
               'val_mean_squared_error': history.history['val_mean_squared_error']})

In [83]:
# Load wights file of the best model :
wights_file = '../logs/weights_model_01/Weights-049--17.80290.hdf5'  # choose the best checkpoint
model.load_weights(wights_file)  # load it
model.compile(loss=keras.losses.MeanSquaredError(
), optimizer=keras.optimizers.Adam(), metrics=[keras.metrics.MeanSquaredError()])

In [84]:
model.summary(expand_nested=True)

In [85]:
for i, layer in enumerate(model.layers):
    print(i, layer)
    try:
        print("    ", layer.activation)
    except AttributeError:
        print('   no activation attribute')

# Check accuracy

In [86]:
score = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [76]:
def to_spherical(coor):
    vector = np.array(
        [(coor[3]-coor[0]), (coor[4]-coor[1]), (coor[5]-coor[2])])
    vectorsphere = np.array([np.sqrt((vector**2).sum()), (np.arctan(np.sqrt(
        coor[0]**2 + coor[1]**2)/coor[2])*180/np.pi), (np.arctan(coor[1]/coor[0])*180/np.pi)])
    return vectorsphere

In [77]:
xtr = model.predict(X_train.iloc[:1])
xtr

In [19]:
to_spherical(np.array(xtr)[0])

In [20]:
np.array(xtr)[0]

In [21]:
ytr = y_train.iloc[:1]
ytr

In [22]:
to_spherical(np.array(ytr)[0])

In [23]:
xtst = model.predict(X_test.iloc[:1])
xtst

In [24]:
ytst = y_test.iloc[:1]
ytst

In [25]:
np.array(ytst)[0]

In [26]:
to_spherical(np.array(xtst)[0])

In [27]:
to_spherical(np.array(ytst)[0])

In [28]:
xtst = model.predict(X_test)
np.array(xtst[0])

In [29]:
x = np.zeros(shape=(len(xtst), 3))

for i in range(len(xtst)):
    x[i] = to_spherical(np.array(xtst[i]))

x_theta = np.zeros(shape=(len(xtst)))
x_phi = np.zeros(shape=(len(xtst)))

for i in range(len(xtst)):
    x_theta[i] = x[i][1]
    x_phi[i] = x[i][2]

In [30]:
np.array(y_test.iloc[0])

In [31]:
y = np.zeros(shape=(len(y_test), 3))

for i in range(len(y_test)):
    y[i] = to_spherical(np.array(y_test.iloc[i]))

y_theta = np.zeros(shape=(len(y_test)))
y_phi = np.zeros(shape=(len(y_test)))

for i in range(len(y_test)):
    y_theta[i] = y[i][1]
    y_phi[i] = y[i][2]

In [32]:
sb.kdeplot((y_theta - x_theta))
plt.ylabel("количество событий, *20")
plt.xlabel("разность предсказаний зенитного угла, радианы")

In [33]:
sb.kdeplot((y_phi - x_phi))
plt.ylabel("количество событий, *20")
plt.xlabel("разность предсказаний азимутального угла, радианы")

In [34]:
plt.hist((y_theta - x_theta))
plt.ylabel("количество событий")
plt.xlabel("разность предсказаний зенитного угла, градусы")
plt.show()

In [35]:
plt.hist((y_phi - x_phi))
plt.ylabel("количество событий")
plt.xlabel("разность предсказаний азимутального угла, градусы")
plt.show()

In [36]:
(y_phi-x_phi).mean()

In [37]:
(y_theta-x_theta).mean()

In [38]:
sb.kdeplot({'Азимутальный': (y_phi - x_phi),
           'Зенитный': (y_theta - x_theta)}, cumulative=False)
plt.ylabel("Значение функции распределения")
plt.xlabel("разность предсказания угла, градусы")

In [39]:
np.std(y_phi - x_phi)

In [40]:
np.std(y_theta - x_theta)

In [41]:
y_df = np.zeros(shape=(len(df_y), 3))

for i in range(len(df_y)):
    y_df[i] = to_spherical(np.array(df_y.iloc[i]))

y_theta_df = np.zeros(shape=(len(df_y)))
y_phi_df = np.zeros(shape=(len(df_y)))

for i in range(len(df_y)):
    y_theta_df[i] = y_df[i][1]
    y_phi_df[i] = y_df[i][2]

In [42]:
ymt = y_theta_df.mean()
ymt

In [43]:
y_theta_df.max()

In [44]:
ymp = y_phi_df.mean()
ymp

In [45]:
y_phi_df.min()

In [46]:
tst = (np.abs(y_phi - x_phi)/np.abs(x_phi))
tst[tst < 1.0].mean()

In [47]:
(np.abs(y_theta - x_theta)/np.abs(x_theta)).mean()

In [48]:
from scipy.stats import shapiro
from scipy.stats import normaltest

In [49]:
shapiro(y_phi - x_phi)

In [50]:
shapiro(y_theta - x_theta)

In [51]:
normaltest(y_phi - x_phi)

In [52]:
normaltest(y_theta - x_theta)

In [64]:
tmp = (y_phi - x_phi)
tmp = tmp[np.abs(tmp) < 30.0]
tmp = pd.DataFrame(tmp, columns=["Азимутальный угол"])
tmp

In [65]:
sb.boxplot(tmp, whis=2.0, fliersize=6)
plt.ylabel("Разность предсказанных и истинных значений")

In [66]:
tmp = (y_theta - x_theta)
tmp = tmp[np.abs(tmp) < 30.0]
tmp = pd.DataFrame(tmp, columns=["Зенитный угол"])
tmp

In [67]:
sb.boxplot(tmp, whis=2.0, fliersize=6)
plt.ylabel("Разность предсказанных и истинных значений")